#                       Failed Circuit Implementations  


#### This circuit was the attempt to create the quantum circuit optimised for number of gates given in supplimentary material of paper "Classically verifiable Quantum advantage from a Computational Bell Test" But unfortunately it was a failure. Our many failed attempts are just given here. We are beginners in this field and still learning about coding and quantum computing but we have put great efforts in attempting to do this

In [49]:
from classiq import *
import numpy as np

@qfunc
def cxfunc(e1:QBit,e2:QBit,e3:QBit)-> None:
    c= QArray("c")
    bind([e1,e2],c)
    control(c,lambda: X(e3))
    bind(c,[e1,e2])
    control(e1,lambda: X(e2))
    X(e1)

    

@qfunc
def ccphazee(l:QBit, p:QBit, o1:QArray[QBit] )-> None:
    ctrl = QArray("ctrl")
    bind([l,p],ctrl)
    control(ctrl ,lambda: cxfunc(o1[0],o1[1],o1[2]))
    
    bind(ctrl,[l,p])

@qfunc
def applyRphi(h: CInt , N: CInt, p: QArray[QBit], g: QBit)  -> None:
 
    repeat(
        count= p.len ,
        iteration=lambda j: CPHASE(((2*(np.pi)*(2**(j+h+1)))/ N),p[j] ,g)
        
    )
    
                   

num=7

@qfunc 
def main(x:Output[QArray[QBit]], y:Output[QArray[QBit]]) -> None : 
    counter = QArray("counter")
    allocate(8,x)
    allocate(10,y)
    allocate(3,counter)
    hadamard_transform(x)
    hadamard_transform(y)
    repeat(count=2*x.len-1,
           iteration=lambda l:repeat(count=x.len,
                                     iteration=lambda i:repeat(count=x.len-(i+1),
                                                               iteration=lambda j: if_(
                                                                                      condition= l==(i+j),
                                                                                      then= lambda: ccphazee(x[i],x[j+1+i],counter),
                                                                                      else_= lambda: IDENTITY(counter[0]),
                                                                                   )
                                                        )
                                )
    
    )
    
  
    repeat(
        count= y.len ,
        iteration=lambda i: applyRphi(i,num,counter,y[i]) 
        )
    invert(lambda: qft(y) )
    

qmod = create_model(main)
quantum_program = synthesize(qmod)
show(quantum_program )
results = execute(quantum_program ).result()
res = results[0].value.counts
res1 = results[0].value.parsed_counts


Opening: https://platform.classiq.io/circuit/ce965682-3ca7-434e-a60b-309286c3c0d9?version=0.44.0


This was a try to implement the circuit given as in main paper "Classically verifiable Quantum advantage from a Computational Bell Test"

In [47]:
from classiq import *
import numpy as np



@qfunc
def ccphazee(N2:CInt, d1:CInt , f1: CInt, t1: CInt, l11:QBit, l22:QBit, o1:QBit )-> None:
    ctrl= QArray("ctrl")
    
    bind([l11,l22],ctrl)
    
    control(ctrl,lambda: PHASE(2*(np.pi)*((2**((d1+1)+(f1+1)+(t1+1)))/ N2), o1) )
    
    bind(ctrl,[l11,l22])
    
    
    
    


@qfunc
def applygates(N1:CInt, p: QArray[QBit], g:QArray[QBit]) -> None:
 
    repeat(
        count= p.len-1,
        iteration=lambda i: repeat(count= p.len-(i+1),
                                   iteration=lambda j: repeat(count= g.len-1,
                                                              iteration=lambda k: if_(
                                                                                     condition= j==0,
                                                                                     then= lambda: CPHASE(2*(np.pi)*((2**((i+1)+(j+1)+(k+1)))/ N1),p[i+1] , g[k+1]),
                                                                                     else_= lambda: ccphazee(N1, i, j,k, p[i+1],p[j+1+i],g[k+1]),
                                                                                    )  
                                                          )
                                   
                                )
        
       )
    
                                
        
    
    
    

@qfunc 
def main(x:Output[QArray[QBit]], y:Output[QArray[QBit]]) -> None : 
    N=7
    allocate(3,x)
    allocate(3,y)
    hadamard_transform(x)
    within_apply(lambda: qft(y), lambda: applygates(N,x,y))
    
qmod = create_model(main)
quantum_program = synthesize(qmod)
show(quantum_program)    
    

Opening: https://platform.classiq.io/circuit/91f51b45-c491-4224-9a10-3b0d3e9c8e3a?version=0.44.0


[This is the conversion of the cirq code given in the circuit folder of github repo available with the paper "Classically verifiable Quantum advantage from a Computational Bell Test",only one part of the code was tried to be implemented-the narrow circuit](https://github.com/GregDMeyer/quantum-advantage)

In [ ]:

from classiq import *
import numpy as np

@qfunc
def cphaze(i1:CInt, j1:CInt, q:QBit, f:QBit) -> None :
    phase = -(1/2**(((j1)+1+(i1))-(i1)))
    control(f, lambda: PHASE((np.pi*phase),q))

    

@qfunc
def phase_add(i2:CInt, j2:CInt, N2:CInt, target:QBit, con:QBit, bi2:CInt) -> None :
    phase1 = 1*2*2**(i2+j2+(bi2))/(N2)
    repeat(
          count= 1,
          iteration= lambda i:control(con, lambda: PHASE((np.pi*(2**((1) - i - 1)) * (phase1)),target)))
    
    
        
@qfunc
def phase_add1(i3:CInt, j3:CInt, N3:CInt, target1:QBit, con1:QBit, con2:QBit, bi3:CInt) -> None :
    phase2 = 2*2*2**(i3+j3+(bi3))/(N3)
    ctrl=QArray("ctrl")
    bind([con1,con2],ctrl)
    repeat(
          count= 1,
          iteration= lambda i:control(ctrl, lambda: PHASE((np.pi*(2**((1) - i - 1) * (phase2))),target1))
    
    
    )
    
    bind(ctrl,[con1,con2])

@qfunc
def x2modN_narrow_single(x1:QArray[QBit], y1:QBit, N1:CInt, bi_dx:CInt,inarrow:CInt):
    
    repeat(
          count=x1.len,
          iteration=lambda i: repeat(
                                    count=x1.len,
                                    iteration= lambda j: if_(
                                                            condition=i==j,
                                                            then= lambda: phase_add(i,j,N1,y1,x1[i],bi_dx),
                                                            else_= lambda: phase_add1(i,j,N1,y1,x1[i],x1[j],bi_dx),
                                    
                                    
                                                         )
          
                                  )
        
        
        )                            
    
    
    
    
    

@qfunc
def circuitcall(xcall:QArray[QBit], ycall:QArray[QBit], N_c:CInt, bi_dx_c:CInt, i_call:CInt)->None:
    x2modN_narrow_single(xcall, ycall[i_call], N_c,bi_dx_c,i_call )
    hadamard_transform(ycall[i_call])
    repeat(
          count=ycall.len-(i_call+1),
          iteration=lambda j:cphaze(i_call,j,ycall[j+1+(i_call)],ycall[i_call]),
         )


@qfunc
def main(N:CInt, x:Output[QArray[QBit]], y:Output[QArray[QBit]]) -> None:
    allocate(5,x)
    allocate(5,y)
    hadamard_transform(x)
    hadamard_transform(y)
    N=77
                                                                                 
    repeat(
          count=y.len,
          iteration=lambda i: circuitcall(x, y, N, ((y.len)-(i)-1), i) ,
                    
        )
        
       

               
           


qmod = create_model(main)
quantum_program = synthesize(qmod)
show(quantum_program)

This was an attempt to implement the circuit given in the supplimentary material which was optimised for number of Qubits

In [50]:
from classiq import *
import numpy as np


@qfunc
def ccphazee( d:CInt , f: CInt, t: CInt, l1:QBit, l2:QBit, o:QBit )-> None:
    ctrl= QArray("ctrl")
    
    bind([l1,l2],ctrl)
    
    control(ctrl,lambda: PHASE(2*(np.pi)*((2**((f+1)+(t+1)+1))/ d), o) )
    
    bind(ctrl,[l1,l2])


@qfunc
def applyRphi(h: CInt, N: CInt, p: QArray[QBit], g: QBit) -> None:
 
    repeat(
        count= p.len - (h+1),
        iteration=lambda j: if_ (
                               condition= j==0,
                               then= lambda: CPHASE(2*(np.pi)*((2**((h+1)+(j+1)+1))/ N),p[h+1] ,g),
                               else_= lambda: ccphazee(N, h, j , p[h+1],p[h+j+1],g ),
                            )
        
    )
    
                                
        
    
    
    


    

num=7

@qfunc 
def main(y:Output[QBit]) -> None : 
    x= QArray("x")
    allocate(5,x)
    allocate(1,y)
    hadamard_transform(x)
    hadamard_transform(y)
    
    repeat(
        count= x.len - 1,
        iteration=lambda i: applyRphi((i),num,x,y)
        )
    PHASE((2*(np.pi)*(2**(2)))%(np.pi), y)
    hadamard_transform(y) 
    

qmod = create_model(main)
quantum_program = synthesize(qmod)
show(quantum_program)

Opening: https://platform.classiq.io/circuit/32ff66e5-dec1-4b79-87e0-88b7fc58ac42?version=0.44.0


 We shall actively try to learn and make ourselves capable of implementing these circuit models in the future.